KLASIFIKASI BANJIR
=========

Import data csv
--------


In [4]:
import pandas as pd 
import os 
import re,string

os.chdir('D:\skripsi\data_fix')
Alldata = pd.read_csv('data_fixx.csv',delimiter=';', names = ['Username','Time','Tweet','Sentimen','DUMP'])
Tablesatu = Alldata.drop('DUMP',axis = 1)

df = [Tablesatu] 
TrainingTable = pd.concat(df,ignore_index = True)
#print (TrainingTable)
data = TrainingTable[['Tweet','Sentimen']] 
ListData = [tuple(x) for x in data.values]
#print (len(ListData))
print (data)

                                                  Tweet  Sentimen
0     Banjir? Banjir Yg mana? Ngga ada banjir hanya ...         3
1     Apa ??u dikata  masih ada yg butuh Pesta "@RIF...         2
2     astagfirullah  jakarta banjir  ahok ramein pes...         3
3                     Jakarta suka banjir karena.......         2
4     Mesti semua ulah avatar. Tanah: Longsor banjar...         2
5     #Sabtu    Info Jakarta @infojekardah     :Sabt...         3
6     Karena banya orang jones:D"@Di_Kepoin: #Dikepo...         2
7     @AntiLiberalNews: Batalkan Acara Sambut Tahun ...         2
8     ?DhekhaDekoyy?  @dhekhadedis     :Hujan Deras ...         2
9     #Katanya Muhammadfikhans: yang jomblo kalau ml...         2
10    terlalu banyak bangunan"@NotersSobat: Jakarta ...         2
11    banjir "@Bobotoh_Kepoo: #Gimel jakarta identik...         2
12    Pada tahun 1621 untuk pertama kalinya Batavia ...         2
13    Djarot Sesumbar Banjir di Jakarta Tidak Lama |...         2
14    JAKA

In [5]:
tweets = []
for a,b in ListData:
    tweets.append((a,b))

prepocessing
---------

In [6]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk


emoticon_str = """
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

regex_str = [
    emoticon_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs

    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticon_str+'$', re.VERBOSE | re.IGNORECASE)

    
#Fungsi Preprocessing

import pandas as pd 
import os 
def remove_hastagnall(tweet):
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|((pic\.[^\s]+)))','',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','',tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #Replace number with spaces
    tweet = re.sub(r'(?:(?:\d+,?)+(?:\.?\d+)?)','',tweet)
    #Remove emoticon
    tweet = re.sub(r'^'+emoticon_str+'$','',tweet)
    #Remove \n
    tweet = tweet.replace('\\n',' ')
    #remove duplicate char
    tweet = re.sub(r'([a-z])\1\1+', r'\1', tweet)
    #Remove punctuation
    tweet = re.sub(r'[^\w\s]',' ',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet

def stemming(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    kata_dasar = stemmer.stem(text)
    return kata_dasar

def tokenize(text):
    return text.split()

def filter_kata(text):
    text = [word for word in text if len(word) >= 3]
    return text

def hapus_stopword(text):
    StopWords = "stopword.txt"
    sw=open(StopWords,encoding='utf-8', mode='r');stop=sw.readlines();sw.close()
    stop=[kata.strip() for kata in stop];stop=set(stop)
    kata = [item for item in text if item not in stop]
    return ' '.join(kata)


simpan prepocessing
----------

In [7]:
import time

start_time = time.time()

tweetbersih = []
for a,b in tweets:
    text = remove_hastagnall(a)
    text2 = stemming(text)
    text3 = tokenize(text2)
    text4 = filter_kata(text3)
    text5 = hapus_stopword(text4)
    tweetbersih.append((text5,b))
    
print("--- %s seconds ---" % (time.time() - start_time))
    
print (tweetbersih)
import csv
with open('prepocessing.csv','w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(tweetbersih)

--- 5103.118670701981 seconds ---
[('banjir banjir ngga banjir air genang metrotipu kabar jakarta banjir parah nggih moga', 3), ('butuh pesta astagfirullah jakarta banjir ahok ramein pesta pake dana', 2), ('astagfirullah jakarta banjir ahok ramein pesta pake dana tega dikau malaysia batalin krn banjir', 3), ('jakarta suka banjir', 2), ('mesti ulah avatar tanah longsor banjarnegara api bakar pasar klewer air banjir jakarta udara jatuh air asia', 2), ('sabtu info jakarta sabtu pagi jakarta timur banjir http', 3), ('banya orang jones poin jakarta suka banjir', 2), ('batal acara sambut malaysia alih dana korban banjir jakarta', 2), ('dhekhadekoyy hujan deras titik banjir jakarta', 2), ('muhammadfikhans jomblo mlm minggu hujanwae jakarta suka banjir', 2), ('bangun jakarta suka banjir', 2), ('banjir gimel jakarta identik', 2), ('kali batavia jakarta alami banjir', 2), ('djarot sesumbar banjir jakarta piyungan online', 2), ('jakarta banjir bilang jokowi ahok bohong pdhal smua salah manusia mu

make vocab & TF-IDF
------------
tf-idf disimpan dalam csv

In [8]:
import pandas as pd 
import numpy as np

from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('prepocessing.csv',delimiter=',', names = ['Tweet','Sentimen'])
tweetbersih = df['Tweet'].tolist()


counter = Counter ()
[counter.update(tweet.split()) for tweet in tweetbersih]
print(counter.most_common(1500))
vocab = counter.most_common(1500)
vocab = [word [:][0] for word in vocab]
print (len(vocab))



def make_tfidf(tweetbersih):
    tfidf = TfidfVectorizer(analyzer='word',min_df=0.01, vocabulary=vocab, ngram_range=(1,1), lowercase=False)
    tfidf_hasil = tfidf.fit_transform(tweetbersih)
    print(tfidf_hasil.toarray())
    return tfidf_hasil

# print(tfidf_hasil.shape)
# a = make_vocab(tweetbersih)
b = make_tfidf(tweetbersih)
print (b.toarray())

np.savetxt('tfidf.csv',b.todense(),delimiter=',')



[('banjir', 8891), ('jakarta', 8450), ('hujan', 1640), ('ahok', 757), ('macet', 660), ('dki', 538), ('air', 532), ('deras', 439), ('guyur', 424), ('kepung', 401), ('gubernur', 382), ('waspada', 366), ('genang', 363), ('bandung', 355), ('pagi', 319), ('warga', 308), ('djarot', 291), ('kampung', 291), ('siaga', 290), ('titik', 289), ('aja', 287), ('jalan', 285), ('kalo', 276), ('bogor', 273), ('musim', 266), ('jokowi', 264), ('gak', 253), ('salah', 251), ('lokasi', 244), ('bebas', 233), ('katulampa', 231), ('berita', 223), ('antisipasi', 222), ('kota', 222), ('udah', 221), ('pulo', 219), ('wagub', 213), ('pohon', 213), ('sabtu', 206), ('hadap', 197), ('timur', 197), ('rendam', 183), ('akibat', 182), ('via', 177), ('wilayah', 174), ('twitter', 172), ('news', 171), ('landa', 169), ('gara', 169), ('rumah', 168), ('tangan', 165), ('moga', 162), ('bantu', 159), ('ujan', 156), ('puncak', 156), ('daerah', 155), ('nih', 153), ('metro', 152), ('presiden', 149), ('barat', 145), ('kali', 140), ('ka

Proses Klasifikasi dengan metode
----------------

import data .csv dan import tfidf.csv

In [9]:
import pandas as pd
import numpy as np
import warnings

# from sklearn import cross_validation
# from sklearn import svm
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.cross_validation import KFold
# from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
warnings.filterwarnings('ignore')

df = pd.read_csv('D:\skripsi\data_fix\data_fixx.csv', delimiter=';', names = ['Username','Time','Tweet','Sentimen','DUMP'])
y = df.Sentimen.values
print(y)

X = pd.read_csv("tfidf.csv", header = None )
print (X)


[3 2 3 ..., 2 2 2]
          0         1         2         3         4         5         6     \
0     0.260704  0.065873  0.000000  0.000000  0.000000  0.000000  0.246407   
1     0.056522  0.057127  0.000000  0.190973  0.000000  0.000000  0.000000   
2     0.136073  0.068765  0.000000  0.229876  0.000000  0.000000  0.000000   
3     0.187259  0.189263  0.000000  0.000000  0.000000  0.000000  0.000000   
4     0.037627  0.038029  0.000000  0.000000  0.000000  0.000000  0.284505   
5     0.068886  0.139247  0.000000  0.000000  0.000000  0.000000  0.000000   
6     0.091744  0.092726  0.000000  0.000000  0.000000  0.000000  0.000000   
7     0.056689  0.057295  0.000000  0.000000  0.000000  0.000000  0.000000   
8     0.150293  0.151901  0.397430  0.000000  0.000000  0.000000  0.000000   
9     0.078180  0.079017  0.000000  0.000000  0.000000  0.000000  0.000000   
10    0.122714  0.124027  0.000000  0.000000  0.000000  0.000000  0.000000   
11    0.703334  0.710860  0.000000  0.000000 

METODE SVM
-------
menggunakan validasi K-cross validation dengan K=10

In [10]:
from sklearn.cross_validation import KFold
from sklearn import svm
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
start_time = time.time()

def evaluasisvm(X,y):
    kf = KFold(y.shape[0], n_folds=10, shuffle=True, random_state=999)
    model_train_index = []
    model_test_index = []
    model = 0
    nilai_akurasisvm = []
    nilai_f1svm =[]
    nilai_precisionsvm =[]
    nilai_recallsvm = []
    
    for k, (index_train, index_test) in enumerate(kf):
        X_train, X_test, y_train, y_test = X.ix[index_train,:], X.ix[index_test,:], y[index_train], y[index_test] 
        clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        f1score = f1_score(y_test, clf.predict(X_test))
        precision = precision_score(y_test, clf.predict(X_test))
        recall = recall_score(y_test, clf.predict(X_test))
        
        nilai_akurasisvm.append(score) #untuk mean
        nilai_f1svm.append(f1score)
        nilai_precisionsvm.append(precision)
        nilai_recallsvm.append(recall)
        
        print('Model %d has accuracy %f with | f1score: %f | precision: %f | recall: %f'%(k,score, f1score, precision, recall))

        model_train_index.append(index_train)
        model_test_index.append(index_test) 
        model+=1
    return score, f1score, precision, recall, nilai_akurasisvm, nilai_f1svm, nilai_precisionsvm, nilai_recallsvm

score, f1score, precision, recall, nilai_akurasisvm, nilai_f1svm, nilai_precisionsvm, nilai_recallsvm = evaluasisvm(X,y)
# evaluasisvm(X,y)
print("--- %s seconds ---" % (time.time() - start_time))

mean_akurasisvm = 0
for a in nilai_akurasisvm:
    mean_akurasisvm = mean_akurasisvm + a
print ('rata-rata nilai akurasi svm adalah %f' %(mean_akurasisvm/10))

mean_f1score = 0
for a in nilai_f1svm:
    mean_f1score = mean_f1score + a
print ('rata-rata nilai f1 svm adalah %f'%(mean_f1score/10))

mean_precision = 0
for a in nilai_precisionsvm:
    mean_precision = mean_precision + a
print ('rata-rata nilai precision svm adalah %f'%(mean_precision/10))

mean_recall = 0
for a in nilai_recallsvm:
    mean_recall = mean_recall + a
print ('rata-rata nilai recall svm adalah %f'%(mean_recall/10))

Model 0 has accuracy 0.796154 with | f1score: 0.797092 | precision: 0.799079 | recall: 0.796154
Model 1 has accuracy 0.797436 with | f1score: 0.796884 | precision: 0.796864 | recall: 0.797436
Model 2 has accuracy 0.816667 with | f1score: 0.815436 | precision: 0.817164 | recall: 0.816667
Model 3 has accuracy 0.780769 with | f1score: 0.779933 | precision: 0.781090 | recall: 0.780769
Model 4 has accuracy 0.803846 with | f1score: 0.803781 | precision: 0.805023 | recall: 0.803846
Model 5 has accuracy 0.794872 with | f1score: 0.793918 | precision: 0.793847 | recall: 0.794872
Model 6 has accuracy 0.793590 with | f1score: 0.791997 | precision: 0.792054 | recall: 0.793590
Model 7 has accuracy 0.807692 with | f1score: 0.805924 | precision: 0.807524 | recall: 0.807692
Model 8 has accuracy 0.807445 with | f1score: 0.806499 | precision: 0.808529 | recall: 0.807445
Model 9 has accuracy 0.777920 with | f1score: 0.777759 | precision: 0.777958 | recall: 0.777920
--- 743.4066321849823 seconds ---
rata-r

METODE Multinominal Naive Bayes
-------
menggunakan validasi K-cross validation dengan K=10
hasilnya akurasi, f1score, precision, dan recall


In [8]:
from sklearn.cross_validation import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
start_time = time.time()

def evaluasiNB(X,y):
    kf = KFold(y.shape[0], n_folds=10, shuffle=True, random_state=9999)
    model_train_index = []
    model_test_index = []
    model = 0
    nilai_akurasiNB = []
    nilai_f1NB =[]
    nilai_precisionNB =[]
    nilai_recallNB = []
    
    
    for k, (index_train, index_test) in enumerate(kf):
        X_train, X_test, y_train, y_test = X.ix[index_train,:], X.ix[index_test,:], y[index_train], y[index_test] 
        clf = MultinomialNB(alpha=1,fit_prior=True, class_prior=None).fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        f1score = f1_score(y_test, clf.predict(X_test))
        precision = precision_score(y_test, clf.predict(X_test))
        recall = recall_score(y_test, clf.predict(X_test))
        
        nilai_akurasiNB.append(score) #untuk mean
        nilai_f1NB.append(f1score)
        nilai_precisionNB.append(precision)
        nilai_recallNB.append(recall)
        
        print('Model %d has accuracy %f with | f1score: %f | precision: %f | recall: %f'%(k,score, f1score, precision, recall))

        model_train_index.append(index_train)
        model_test_index.append(index_test) 
        model+=1
    return score, f1score, precision, recall, nilai_akurasiNB, nilai_f1NB, nilai_precisionNB, nilai_recallNB

score, f1score, precision, recall, nilai_akurasiNB, nilai_f1NB, nilai_precisionNB, nilai_recallNB = evaluasiNB(X,y)
# evaluasiNB(X,y)

print("--- %s seconds ---" % (time.time() - start_time))
mean_akurasiNB = 0
for a in nilai_akurasiNB:
    mean_akurasiNB = mean_akurasiNB + a
print ('rata-rata nilai akurasi Multinominal Naive bayes adalah %f' %(mean_akurasiNB/10))

mean_f1score = 0
for a in nilai_f1NB:
    mean_f1score = mean_f1score + a
print ('rata-rata nilai f1 Multinominal Naive bayes adalah %f'%(mean_f1score/10))

mean_precision = 0
for a in nilai_precisionNB:
    mean_precision = mean_precision + a
print ('rata-rata nilai precision Multinominal Naive bayes adalah %f'%(mean_precision/10))

mean_recall = 0
for a in nilai_recallNB:
    mean_recall = mean_recall + a
print ('rata-rata nilai recall Multinominal Naive bayes adalah %f'%(mean_recall/10))

Model 0 has accuracy 0.820000 with | f1score: 0.808641 | precision: 0.830294 | recall: 0.820000
Model 1 has accuracy 0.840000 with | f1score: 0.832626 | precision: 0.836945 | recall: 0.840000
Model 2 has accuracy 0.810000 with | f1score: 0.798794 | precision: 0.822368 | recall: 0.810000
Model 3 has accuracy 0.810000 with | f1score: 0.789652 | precision: 0.822575 | recall: 0.810000
Model 4 has accuracy 0.770000 with | f1score: 0.758552 | precision: 0.769903 | recall: 0.770000
Model 5 has accuracy 0.790000 with | f1score: 0.782422 | precision: 0.792841 | recall: 0.790000
Model 6 has accuracy 0.840000 with | f1score: 0.834841 | precision: 0.847221 | recall: 0.840000
Model 7 has accuracy 0.790000 with | f1score: 0.781511 | precision: 0.790893 | recall: 0.790000
Model 8 has accuracy 0.780000 with | f1score: 0.760229 | precision: 0.800492 | recall: 0.780000
Model 9 has accuracy 0.800000 with | f1score: 0.789016 | precision: 0.830400 | recall: 0.800000
--- 0.6663413047790527 seconds ---
rata-

## Perbandingan akurasi dalam grafik
------------------

In [14]:
import matplotlib.pyplot as plt
plt.plot(nilai_akurasisvm, 'r')
plt.plot(nilai_akurasiNB, 'g')
plt.show()

grafik hasil
!(D:\skripsi\figure.png)

TESTING DATA BARU
============

membuat model klasifikasi SVM (termasuk TF-IDF) untuk testing
-------------
model klasifikasi disimpan dalam pickle

In [11]:
from sklearn import svm
from sklearn.svm import SVR
# from sklearn.naive_bayes import MultinominalNB
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
import pandas as pd
import pickle


def model_classifier(x,y):
    clf = svm.SVC(kernel='linear', C=1)
#     clf = MultinomialNB(alpha=1,fit_prior=True, class_prior=None).fit(X_train, y_train)
    tfidf = TfidfVectorizer(analyzer='word',min_df=0.01, ngram_range=(1,1), lowercase=False)
    tfidf_svm = Pipeline([('tfidf',tfidf), ('evaluasisvm',clf)])
    tfidf_svm.fit(x,y)
    svm_dump = pickle.dumps(tfidf_svm)
    return svm_dump




Input Data baru
-------------

Data baru dilakukan prepocessing

In [12]:
import time
import os
import pandas as pd
from sklearn.externals import joblib
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk

if __name__ == '__main__':
    print ('Hello , ini identifikasi banjir')
    print ('==============================')
    time.sleep(1)
    user_input = input("masukkan path dari file: ")
    assert os.path.exists(user_input), 'I did not find the file at, '+str(user_input)
   
    databaru = pd.read_csv(user_input, sep=",", header=0, names = ['Time', 'Tweet'])
#     TrainingTablebaru = pd.concat(f,ignore_index = True)
#     Tablesatu = databaru.drop('Username',axis = 1)
    Tablesatu = databaru.drop('Time',axis=1)
    data = databaru['Tweet']
#     print (data)
    data.values.tolist()
    ListData = data.values.tolist()
#     ListData = [tuple(x) for x in data.values]
  
    tweetbersihbaru = []
    for a in ListData:
        text = remove_hastagnall(a)
        text2 = stemming(text)
        text3 = tokenize(text2)
        text4 = filter_kata(text3)
        text5 = hapus_stopword(text4)
        tweetbersihbaru.append((text5))
    print(tweetbersihbaru)


# D:\skripsi\data_fix\hasiltesting.csv

Hello , ini identifikasi banjir
masukkan path dari file: D:\skripsi\crawling\search_twittertesting4.csv
['jum waves dimitri vegas like mike tsadiera banjir kanal timur bkt', 'kawasan tanggul kali banjir kanal barat kota bambu utara jakbar kondusif', 'kagak ujan bees banjir banjir patwa vivi phardie bintang sempurna print jam benhil pic', 'derita anak taman duta balada banjir kalo gak tidur cuci mobil rumah taman duta pic', 'jabat condet gimane jalan kaya celiwung banjir condet celilitan pgc', 'banjir lalin macet ciwastra bandung jabar via', 'banjir dusun leng kec jagoibabang kab bengkayang surut', 'berisik yak nyata nyata gue banjir love niih jan postingan minimal loves', 'tau ujan mulu kasi bocor mending bocor doang mah ampe banjir', 'kbun binatang banjir datengin jngan lupa dateng theater kebun binatang hujan', 'banjir bwt sembuh bapa alhamdulillah besok udh plg with sandy sari asih rahmah', 'ngono sik folow luwih berisik rituitanmu kadang alir koyo banjir ngebaki jokowi berisik', 'm

pemanggilan model klasifikasi yang telah disimpan 
-----------------------

In [13]:
# f[tweetbersihbaru]
start_time = time.time()

b = pickle.loads(model_classifier(tweetbersih,y))
result = b.predict(tweetbersihbaru)
print ("ini hasilnya : " +str(result))

print("--- %s seconds ---" % (time.time() - start_time))

ini hasilnya : [2 3 3 ..., 2 2 2]
--- 2.912213087081909 seconds ---


Hasil akhir dari data testing
------------------

In [14]:
import numpy as np

df_result_svm = pd.DataFrame(result)
final_svm_predict = pd.concat([databaru,df_result_svm],axis=1)
final_svm_predict.columns = ['Time','Tweet', 'Sentimen']
print(final_svm_predict)

final_svm_predict['Sentimen'].value_counts().sort_values()


# np.savetxt('hasiltesting.csv',final_svm_predict.toarray(),delimiter=',')
# with open('hasiltesting.csv','w', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerows(final_svm_predict)
df = pd.DataFrame(final_svm_predict)
filename = 'hasiltesting2.csv'
df.to_csv(filename, index=False, encoding='utf-8')

             Time                                              Tweet  Sentimen
0     29 Jan 2015  Jum'at ? Waves by Dimitri Vegas  Like Mike & W...         2
1     29 Jan 2015  Kawasan tanggul Kali Banjir Kanal Barat RW 02 ...         3
2     29 Jan 2015  Kagak ujan tp bees kok banjir banjir? (w/ Patw...         3
3     29 Jan 2015  Derita anak taman duta balada banjir. Kalo gak...         2
4     29 Jan 2015  Pejabat condet gimane ni tiap hari jalanan kay...         3
5     29 Jan 2015  RT"@RadioElshinta: 19.33: Banjir menyebabkan l...         3
6     29 Jan 2015  Banjir di dusun peleng kec. Jagoibabang kab. B...         2
7     29 Jan 2015  Ini HP berisik amat yak.\nTernyata eh ternyata...         2
8     29 Jan 2015  @Kartika_KPR tau ujan mulu kasian bocor masih ...         3
9     28 Jan 2015  Ada kbun binatang saat banjir ga? Mau gw daten...         3
10    28 Jan 2015  Banjir do'a bwt kesembuhan bapa :) alhamdulill...         2
11    28 Jan 2015  Ngono sik folow :) "@kokokdirgant